In [68]:
from nba_api.stats.endpoints import playercareerstats
import pandas as pd
from nba_api.stats.static import players
from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.library.parameters import Season
from nba_api.stats.library.parameters import SeasonType
from nba_api.stats.library.parameters import SeasonTypePlayoffs

import json


In [75]:
def json_to_file(f_name, sometext):
    json_object = json.dumps(sometext, indent=4)
    with open(f'data/{f_name}.json', "w") as outfile:
        outfile.write(json_object)


In [76]:
nba_teams = teams.get_teams()

json_to_file("nba_teams",nba_teams)

In [77]:
team_abb_collection = []
for i in teams.get_teams():
    team_abb_collection.append(i['abbreviation'])

team_abb_collection

['ATL',
 'BOS',
 'CLE',
 'NOP',
 'CHI',
 'DAL',
 'DEN',
 'GSW',
 'HOU',
 'LAC',
 'LAL',
 'MIA',
 'MIL',
 'MIN',
 'BKN',
 'NYK',
 'ORL',
 'IND',
 'PHI',
 'PHX',
 'POR',
 'SAC',
 'SAS',
 'OKC',
 'TOR',
 'UTA',
 'MEM',
 'WAS',
 'DET',
 'CHA']

In [78]:
# Select the dictionary for the Team (BOS, LAL, IND), which contains their team ID 
team_abb = 'DAL'
team_select = [team for team in nba_teams if team['abbreviation'] == team_abb][0]
team_select

{'id': 1610612742,
 'full_name': 'Dallas Mavericks',
 'abbreviation': 'DAL',
 'nickname': 'Mavericks',
 'city': 'Dallas',
 'state': 'Texas',
 'year_founded': 1980}

In [79]:
f'team_id: {team_select["id"]}'

'team_id: 1610612742'

In [80]:
# export player to file
all_player_list = players.get_players()
json_to_file("all_player",all_player_list)

In [82]:
# Search for player with name
player_name = "Bol"
player_result = []
for i in all_player_list:
    if i['first_name'] == player_name:
        player_result.append(i)
    elif i['last_name'] == player_name:
        player_result.append(i)
    
print(player_result)

[{'id': 1629626, 'full_name': 'Bol Bol', 'first_name': 'Bol', 'last_name': 'Bol', 'is_active': True}, {'id': 76195, 'full_name': 'Manute Bol', 'first_name': 'Manute', 'last_name': 'Bol', 'is_active': False}]


In [5]:
# Nikola Jokić
career = playercareerstats.PlayerCareerStats(player_id='203999') 


In [8]:
# career.get_json()
career.get_dict()

{'resource': 'playercareerstats',
 'parameters': {'PerMode': 'Totals', 'PlayerID': 203999, 'LeagueID': None},
 'resultSets': [{'name': 'SeasonTotalsRegularSeason',
   'headers': ['PLAYER_ID',
    'SEASON_ID',
    'LEAGUE_ID',
    'TEAM_ID',
    'TEAM_ABBREVIATION',
    'PLAYER_AGE',
    'GP',
    'GS',
    'MIN',
    'FGM',
    'FGA',
    'FG_PCT',
    'FG3M',
    'FG3A',
    'FG3_PCT',
    'FTM',
    'FTA',
    'FT_PCT',
    'OREB',
    'DREB',
    'REB',
    'AST',
    'STL',
    'BLK',
    'TOV',
    'PF',
    'PTS'],
   'rowSet': [[203999,
     '2015-16',
     '00',
     1610612743,
     'DEN',
     21.0,
     80,
     55,
     1733.0,
     307,
     600,
     0.512,
     28,
     84,
     0.333,
     154,
     190,
     0.811,
     181,
     379,
     560,
     189,
     79,
     50,
     104,
     208,
     796],
    [203999,
     '2016-17',
     '00',
     1610612743,
     'DEN',
     22.0,
     73,
     59,
     2038.0,
     494,
     854,
     0.578,
     45,
     139,
     0.

In [90]:
gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=team_select['id'],
                            season_nullable=Season.default,
                            season_type_nullable=SeasonType.regular
                            # season_type_nullable=SeasonTypePlayoffs.playoffs
                                              )  

games_dict = gamefinder.get_normalized_dict()
print(SeasonTypePlayoffs.playoffs)

Playoffs


In [91]:
match_up_team = "BOS"
matchup_result = []
for i in games_dict['LeagueGameFinderResults']:
    if match_up_team in i['MATCHUP']:
    # if match_up_team.MATCH.str.contsins('MIN:
        matchup_result.append(i)

json_to_file("match_result",matchup_result)

        

In [92]:
games = games_dict['LeagueGameFinderResults']
game = games[0]
game_id = game['GAME_ID']
game_matchup = game['MATCHUP']

# print(f'Searching through {len(games)} game(s) for the game_id of {game_id} where {game_matchup}')
print(game)

{'SEASON_ID': '22023', 'TEAM_ID': 1610612742, 'TEAM_ABBREVIATION': 'DAL', 'TEAM_NAME': 'Dallas Mavericks', 'GAME_ID': '0022301196', 'GAME_DATE': '2024-04-14', 'MATCHUP': 'DAL @ OKC', 'WL': 'L', 'MIN': 241, 'PTS': 86, 'FGM': 32, 'FGA': 97, 'FG_PCT': 0.33, 'FG3M': 8, 'FG3A': 39, 'FG3_PCT': 0.205, 'FTM': 14, 'FTA': 17, 'FT_PCT': 0.824, 'OREB': 12, 'DREB': 29, 'REB': 41, 'AST': 19, 'STL': 8, 'BLK': 2, 'TOV': 14, 'PF': 15, 'PLUS_MINUS': -49.0}


In [100]:
# Get Playbyplay
from nba_api.stats.endpoints import playbyplayv2
pbp = playbyplayv2.PlayByPlayV2('0022301196')
pbp = pbp.get_data_frames()[0]
pbp.head()

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,...,PLAYER2_TEAM_NICKNAME,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION,VIDEO_AVAILABLE_FLAG
0,0022301196,2,12,0,1,3:41 PM,12:00,None,Start of 1st Period (3:41 PM EST),None,...,None,None,0,0,None,NaN,None,None,None,0
1,0022301196,4,10,0,1,3:41 PM,12:00,,None,None,...,Mavericks,DAL,3,1610612742,None,NaN,None,None,None,1
2,0022301196,8,2,75,1,3:42 PM,11:37,None,None,MISS Hardaway Jr. 4' Driving Finger Roll Layup,...,None,None,0,0,None,NaN,None,None,None,1
3,0022301196,10,4,0,1,3:42 PM,11:34,Holmgren REBOUND (Off:0 Def:1),None,None,...,None,None,0,0,None,NaN,None,None,None,1
4,0022301196,11,10,0,1,3:43 PM,11:14,Jump Ball Holmgren vs. Prosper: Tip to Giddey,None,None,...,Mavericks,DAL,4,1630581,Josh Giddey,1.610613e+09,Oklahoma City,Thunder,OKC,1


In [101]:
# Get BoxScore
from nba_api.live.nba.endpoints import boxscore
box = boxscore.BoxScore('0022301196') 

# Data Sets
# box.game.get_dict()                  #equal to box.get_dict()['game']
#box.arena.get_dict()                  #equal to box.get_dict()['game']['arena']
#box.away_team.get_dict()              #equal to box.get_dict()['game']['awayTeam']
#box.away_team_player_stats.get_dict() #equal to box.get_dict()['game']['awayTeam']['players']
#box.away_team_stats.get_dict()        #equal to box.get_dict()['game']['homeTeam'] w/o ['players']
#box.home_team.get_dict()              #equal to box.get_dict()['game']['homeTeam']
#box.home_team_player_stats.get_dict() #equal to box.get_dict()['game']['homeTeam']['players']
#box.home_team_stats.get_dict()        #equal to box.get_dict()['game']['homeTeam'] w/o ['players']
#box.game_details.get_dict()           #equal to box.get_dict()['game'] scrubbed of all other dictionaries
#box.officials.get_dict()              #equal to box.get_dict()['game']['officials']

In [99]:
box.away_team_player_stats.get_dict()  

[{'status': 'ACTIVE',
  'order': 1,
  'personId': 1630182,
  'jerseyNum': '8',
  'position': 'SF',
  'starter': '1',
  'oncourt': '0',
  'played': '1',
  'statistics': {'assists': 3,
   'blocks': 0,
   'blocksReceived': 0,
   'fieldGoalsAttempted': 8,
   'fieldGoalsMade': 1,
   'fieldGoalsPercentage': 0.125,
   'foulsOffensive': 0,
   'foulsDrawn': 3,
   'foulsPersonal': 0,
   'foulsTechnical': 0,
   'freeThrowsAttempted': 2,
   'freeThrowsMade': 2,
   'freeThrowsPercentage': 1.0,
   'minus': 89.0,
   'minutes': 'PT31M17.03S',
   'minutesCalculated': 'PT31M',
   'plus': 45.0,
   'plusMinusPoints': -44.0,
   'points': 4,
   'pointsFastBreak': 0,
   'pointsInThePaint': 2,
   'pointsSecondChance': 0,
   'reboundsDefensive': 2,
   'reboundsOffensive': 1,
   'reboundsTotal': 3,
   'steals': 0,
   'threePointersAttempted': 4,
   'threePointersMade': 0,
   'threePointersPercentage': 0.0,
   'turnovers': 1,
   'twoPointersAttempted': 4,
   'twoPointersMade': 1,
   'twoPointersPercentage': 0.25